In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
Vegas = pd.read_csv('vegas.csv')
Vegas.columns = Vegas.columns.str.replace('\.*\s+', '_').str.lower()

# General Split-Apply-Combine

Remember the basic workflow of split-apply-combine.

<img src="s-a-c.png">

So far, we've looked at aggregation, which is one example of a split-apply-combine procedure. Aggregation is limited in some ways.  Each group is reduced to a single row of information.  Columns are processed independently, so one variable can't affect the way another is aggregated.

For more general split-apply-combine operations, Pandas gives us the `apply()` method.  With this method, we can perform arbitrary processing on each group, turning into a `Series` or `DataFrame` with any shape.  Whatever the results look like, Pandas will try to concatenate them together vertically.

### Example 1.

Let's see a simple example.  Suppose we want information about the 3 reviewers for each hotel who have the most review-writing experience. Notice that this is not an aggregation, because the result from each group will span three rows.

To begin with, let's group by `hotel_name` and pull out one group we can experiment with.

Next, we need to write a function that takes a group like this as input, and returns information about the 3 most experienced reviewers.  We can use `sort_values` followed by `iloc` to pull out the top three rows.  We'll also keep only the first five columns of the result, to simplify the output.

In [ ]:
def top_reviewers(group):
    return group.sort_values(by = 'nr_reviews', ascending = False).iloc[:3, :5]

Once we write our function, the next step is to test it on our sample group, to see if it works properly.

Finally, let's apply the function to all groups, by passing it into `apply`.

You can see that Pandas applied our function to each group, resulting in a three-row dataframe, then glued these together vertically.  We also get a hierarchical index where level 0 is formed by the group key, so we can tell the chunks apart.

### Example 2.

We previously saw how to aggregate scores into a mean score for each hotel.  To get more detail, we may want to create a table to show the percentage of 5's, the percentage of 4's, and so on.  Imagine putting the hotels on the Y=axis, with a column for each score between 1 and 5.  

Once again, this isn't an aggregation, because a single column of data will turn into 5 columns in the result.  Let's write a custom function and test it on our sample group.

That seems to work, so let's go ahead and apply it to all groups with `apply`.

The result is working, but it's not in the format we want.  Instead of a wide table, we get a tall `DataFrame` with a hierarchical index.  To fix this, we could use `unstack()`.

Notice that there are some missing values, which should be replaced with zero.  We can fix that with `fillna()`.

If you don't like `unstack()`, there is another way to put the data into the right shape.  This way takes longer, but I think it's worth seeing to get a deeper understanding of split-apply-combine.  

Our problem was that our custom function returns a `Series`, so it's not clear if the orientation we want is horizontal or vertical.  We could force it to be the way we want by turning the `Series` into a `DataFrame`.  The `.T` is needed to transpose the result into a single row.

Notice that the order is wrong here, but Python might fix this for us later.  Let's go ahead and apply to all groups.

Notice that we get an extra index at level 1, which we don't want.  We can get rid of this with a call to `reset_index`.  We'll also fill in the missing values at this point.

Let's try turning this into a heatmap.

### Example 3

Suppose you want to know where the reviewers for each hotel are coming from.  You imagine a table where the hotels are on the y-axis, and each column shows the fraction of a hotel's reviewers that come from a given country.  To keep things compact, you want to include only the 10 most common countries.  The final table might begin like this.

  | USA | Cananda | Australia | ...
---|---|---|---|---
Circus Circus |  .53 | .33 | .12 |...
Ballagio | .44 | .35 | .09 | ...

Statistically speaking, this won't be a very meaningful exercise:  We have a very small sample of reviewers, and they may have been selected using a complex procedure, which means that they're not representative of customers as a whole.  Still, it's a great way to show off the power of `groupby`.  

When you have a challenge like this, you have to think backwards.  How can you create each of these rows so that Pandas can paste them together into the final table?

Let's start writing a function to pass into `apply`.  We know that `value_counts()` gives us close to what we want.

If we pass this function into `apply`, we get some good information, but it's in the wrong shape and the countries for each hotel don't match.

We know that we only want to keep the top 10 countries in our final table.  Let's pull those out first.

When we call `value_counts`, we get a `Series` indexed by country, and we need to only keep the countries on this list.  We can do that by bracket indexing with `top_countries`.  We also need to use `fillna()` to account for missing values.

That seems to work on our sample group, so we can go head and apply to all groups.

Let's throw this into a heatmap to look for any prominent features.

In [ ]:
sns.heatmap(by_hotel.apply(country_counts_2), annot = True, cmap = 'Blues', alpha = .8)

Since we have such a small sample, the features we see here are very likely to be noise.  Still, if you imagine that this were a larger and representative sample of hotel customers, you could start to pick out interesting facts - which hotels attract an international crowd and so forth.

Group operations can get a lot more complicated than this, and they often require creativity to pull off.  There are some more good examples of using `apply` in the textbook - I highly recommend you take a look at those.

Finally remember two simple tips.

1. get a sample group to experiment on.
2. work backwards: draw out what you want your final table to look like.  Inspect a single row and figure out how to create it from a group.  (Note: it could be a set of rows, or a column or a set of columns.

## Pair Programming

Answer the following questions

1. How do travel patterns differ by traveller type? Do business travellers frequent different hotels from friends and solo travels?

2. Are there trends in hotel reviews improving or or staying consistent across period of stay or review month?

3. Write a function to group hotels as having "many ammeneties" if the hotel has a pool, spa, and gym, and tennis court. Group the average reviews by those that have "many ammenities" versus "fewer amenties". 

Before you code, Write out (in markdown cell)
* What variables will you group by?
* What function will you apply? 
* What will the resulting shape be of the output? 